In [1]:
import pandas as pd
import requests
import time
import geopandas as gpd
from dateutil.relativedelta import relativedelta
import datetime as dt
import math

In [76]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)

In [29]:
masterlist_file_loc = "../datasets/hdb_resale_flat_address_masterlist.csv"
# masterlist_file_loc = "./temp_masterlist.csv"
masterlist_df = pd.read_csv(
    masterlist_file_loc,
    index_col=0,
    dtype={"postal": "object"},
    parse_dates=["lease_commence_date"],
)

In [23]:
mrt_file_loc = "../datasets/mrt_lrt_stations_2025-01-14.csv"
mrt_df = pd.read_csv(mrt_file_loc, parse_dates=["opening"], index_col=0)
mrt_gdf = gpd.GeoDataFrame(
    mrt_df, geometry=gpd.points_from_xy(mrt_df["x"], mrt_df["y"])
)

mrt_gdf.head(1)

,code,station_name,line,color,opening,type,blk_no,road_name,building,address,postal,x,y,latitude,longitude,planning_area_ura,region_ura,geometry
0,NS1,Jurong East,North-South Line,Red,1990-03-10,MRT,10,JURONG EAST STREET 12,JURONG EAST MRT STATION (EW24 / NS1),10 JURONG EAST STREET 12 JURONG EAST MRT STATI...,609690,17869.057052,35038.96887,1.333153,103.742286,JURONG EAST,WEST REGION,POINT (17869.057 35038.969)


In [87]:
def find_closest_station(row, mrt_gdf):
    """Remove comments for the following 3 lines if you want the closest MRT at the time of transaction"""
    # mrt_gdf["OPENING_DATE"] = mrt_gdf["opening"].dt.to_period("M").dt.to_timestamp()
    # mrt_stations_filtered = mrt_gdf[mrt_gdf["OPENING_DATE"] < row["month"]]
    # distances = mrt_stations_filtered.distance(row["geometry"])

    """Remove comments for this line if you want the closest MRT station today"""
    distances = mrt_gdf.distance(row["geometry"])

    closest_station_index = distances.idxmin()
    shortest_distance = distances.min()

    closest_station_name = mrt_gdf.loc[closest_station_index, "station_name"]
    closest_transport_type = mrt_gdf.loc[closest_station_index, "type"]
    closest_mrt_color = mrt_gdf.loc[closest_station_index, "color"]

    # distance to cbd
    raffles_place_index = mrt_gdf.query("station_name == 'Raffles Place'").index[0]
    distance_to_cbd = mrt_gdf.loc[raffles_place_index, "geometry"].distance(
        row["geometry"]
    )

    return pd.Series(
        [
            closest_station_name,
            shortest_distance,
            closest_transport_type,
            closest_mrt_color,
            distance_to_cbd,
        ],
        index=[
            "closest_mrt_station",
            "distance_to_mrt_meters",
            "transport_type",
            "line_color",
            "distance_to_cbd",
        ],
    )

In [88]:
# Convert DataFrame to GeoDataFrame
masterlist_gdf = gpd.GeoDataFrame(
    masterlist_df, geometry=gpd.points_from_xy(masterlist_df["x"], masterlist_df["y"])
)

masterlist_gdf

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,new_closest_mrt_station,new_distance_to_mrt_meters,geometry
0,ANG MO KIO,205,ANG MO KIO AVE 1,205,ANG MO KIO AVENUE 1,NIL,560205,205 ANG MO KIO AVENUE 1 SINGAPORE 560205,1977-01-01,ANG MO KIO,NORTH-EAST REGION,29142.244275,38774.891527,1.366941,103.843582,Ang Mo Kio,742.768808,MRT,Red,9199.172507,ANG MO KIO PRIMARY SCHOOL,512.545254,Ang Mo Kio,742.768808,POINT (29142.244 38774.892)
1,ANG MO KIO,207,ANG MO KIO AVE 1,207,ANG MO KIO AVENUE 1,ANG MO KIO 22,560207,207 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29060.485578,38651.052977,1.365821,103.842848,Ang Mo Kio,874.305666,MRT,Red,9084.169820,ANG MO KIO PRIMARY SCHOOL,527.275749,Ang Mo Kio,874.305666,POINT (29060.486 38651.053)
2,ANG MO KIO,208,ANG MO KIO AVE 1,208,ANG MO KIO AVENUE 1,ANG MO KIO 22,560208,208 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29045.715075,38609.483079,1.365445,103.842715,Ang Mo Kio,908.966103,MRT,Red,9044.410980,ANG MO KIO PRIMARY SCHOOL,549.157654,Ang Mo Kio,908.966103,POINT (29045.715 38609.483)
3,ANG MO KIO,215,ANG MO KIO AVE 1,215,ANG MO KIO AVENUE 1,ANG MO KIO 22,560215,215 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28924.303291,38732.591142,1.366558,103.841624,Mayflower,781.530965,MRT,Brown,9180.500545,ANG MO KIO PRIMARY SCHOOL,377.594832,Mayflower,781.530965,POINT (28924.303 38732.591)
4,ANG MO KIO,216,ANG MO KIO AVE 1,216,ANG MO KIO AVENUE 1,ANG MO KIO 22,560216,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28911.052240,38692.616791,1.366197,103.841505,Mayflower,800.632187,MRT,Brown,9142.409400,ANG MO KIO PRIMARY SCHOOL,403.611794,Mayflower,800.632187,POINT (28911.052 38692.617)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9659,YISHUN,876,YISHUN ST 81,876,YISHUN STREET 81,NIL,760876,876 YISHUN STREET 81 SINGAPORE 760876,1987-12-01,YISHUN,NORTH REGION,28246.274546,44060.806242,1.414745,103.835532,Khatib,407.136706,MRT,Red,14551.599178,NAVAL BASE PRIMARY SCHOOL,402.052254,Khatib,407.136706,POINT (28246.275 44060.806)
9660,YISHUN,877,YISHUN ST 81,877,YISHUN STREET 81,NIL,760877,877 YISHUN STREET 81 SINGAPORE 760877,1987-12-01,YISHUN,NORTH REGION,28237.634702,43967.636908,1.413902,103.835454,Khatib,473.260140,MRT,Red,14460.190304,NAVAL BASE PRIMARY SCHOOL,457.007677,Khatib,473.260140,POINT (28237.635 43967.637)
9661,YISHUN,878,YISHUN ST 81,878,YISHUN STREET 81,NIL,760878,878 YISHUN STREET 81 SINGAPORE 760878,1988-01-01,YISHUN,NORTH REGION,28285.898091,43984.300584,1.414053,103.835888,Khatib,490.234213,MRT,Red,14470.867011,NAVAL BASE PRIMARY SCHOOL,408.316400,Khatib,490.234213,POINT (28285.898 43984.301)
9662,YISHUN,879,YISHUN ST 81,879,YISHUN STREET 81,NIL,760879,879 YISHUN STREET 81 SINGAPORE 760879,1987-10-01,YISHUN,NORTH REGION,28311.512736,44027.290776,1.414442,103.836118,Khatib,477.228688,MRT,Red,14510.511874,NAVAL BASE PRIMARY SCHOOL,362.256294,Khatib,477.228688,POINT (28311.513 44027.291)


In [89]:
masterlist_df_trunc = masterlist_df.loc[:, "town":"longitude"]
masterlist_gdf_trunc = gpd.GeoDataFrame(
    masterlist_df_trunc, geometry=gpd.points_from_xy(masterlist_df_trunc["x"], masterlist_df_trunc["y"])
)

masterlist_gdf_trunc

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,geometry
0,ANG MO KIO,205,ANG MO KIO AVE 1,205,ANG MO KIO AVENUE 1,NIL,560205,205 ANG MO KIO AVENUE 1 SINGAPORE 560205,1977-01-01,ANG MO KIO,NORTH-EAST REGION,29142.244275,38774.891527,1.366941,103.843582,POINT (29142.244 38774.892)
1,ANG MO KIO,207,ANG MO KIO AVE 1,207,ANG MO KIO AVENUE 1,ANG MO KIO 22,560207,207 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29060.485578,38651.052977,1.365821,103.842848,POINT (29060.486 38651.053)
2,ANG MO KIO,208,ANG MO KIO AVE 1,208,ANG MO KIO AVENUE 1,ANG MO KIO 22,560208,208 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29045.715075,38609.483079,1.365445,103.842715,POINT (29045.715 38609.483)
3,ANG MO KIO,215,ANG MO KIO AVE 1,215,ANG MO KIO AVENUE 1,ANG MO KIO 22,560215,215 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28924.303291,38732.591142,1.366558,103.841624,POINT (28924.303 38732.591)
4,ANG MO KIO,216,ANG MO KIO AVE 1,216,ANG MO KIO AVENUE 1,ANG MO KIO 22,560216,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28911.052240,38692.616791,1.366197,103.841505,POINT (28911.052 38692.617)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9659,YISHUN,876,YISHUN ST 81,876,YISHUN STREET 81,NIL,760876,876 YISHUN STREET 81 SINGAPORE 760876,1987-12-01,YISHUN,NORTH REGION,28246.274546,44060.806242,1.414745,103.835532,POINT (28246.275 44060.806)
9660,YISHUN,877,YISHUN ST 81,877,YISHUN STREET 81,NIL,760877,877 YISHUN STREET 81 SINGAPORE 760877,1987-12-01,YISHUN,NORTH REGION,28237.634702,43967.636908,1.413902,103.835454,POINT (28237.635 43967.637)
9661,YISHUN,878,YISHUN ST 81,878,YISHUN STREET 81,NIL,760878,878 YISHUN STREET 81 SINGAPORE 760878,1988-01-01,YISHUN,NORTH REGION,28285.898091,43984.300584,1.414053,103.835888,POINT (28285.898 43984.301)
9662,YISHUN,879,YISHUN ST 81,879,YISHUN STREET 81,NIL,760879,879 YISHUN STREET 81 SINGAPORE 760879,1987-10-01,YISHUN,NORTH REGION,28311.512736,44027.290776,1.414442,103.836118,POINT (28311.513 44027.291)


In [90]:
new_mrt = masterlist_gdf_trunc.apply(find_closest_station, mrt_gdf=mrt_gdf, axis=1)

In [91]:
masterlist_df[["new_closest_mrt_station", "new_distance_to_mrt_meters"]] = new_mrt.loc[:, "closest_mrt_station":"distance_to_mrt_meters"]

In [94]:
masterlist_df[masterlist_df["closest_mrt_station"] != masterlist_df["new_closest_mrt_station"]].reset_index()

,index,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,new_closest_mrt_station,new_distance_to_mrt_meters
0,714,BEDOK,155,BEDOK STH AVE 3,155,BEDOK SOUTH AVENUE 3,SIGLAP EAST,460155,155 BEDOK SOUTH AVENUE 3 SIGLAP EAST SINGAPORE...,1985-08-01,BEDOK,EAST REGION,40549.349711,33427.021003,1.318574,103.946081,Tanah Merah,952.875213,MRT,Green,11198.038021,TEMASEK PRIMARY SCHOOL,110.332130,Bayshore,734.517932
1,715,BEDOK,156,BEDOK STH AVE 3,156,BEDOK SOUTH AVENUE 3,SIGLAP EAST,460156,156 BEDOK SOUTH AVENUE 3 SIGLAP EAST SINGAPORE...,1985-12-01,BEDOK,EAST REGION,40482.672491,33395.236866,1.318287,103.945482,Tanah Merah,988.907307,MRT,Green,11124.527373,TEMASEK PRIMARY SCHOOL,65.254037,Bayshore,671.333116
2,716,BEDOK,157,BEDOK STH AVE 3,157,BEDOK SOUTH AVENUE 3,SIGLAP EAST,460157,157 BEDOK SOUTH AVENUE 3 SIGLAP EAST SINGAPORE...,1985-04-01,BEDOK,EAST REGION,40508.693332,33444.935079,1.318736,103.945716,Tanah Merah,937.145118,MRT,Green,11165.942698,TEMASEK PRIMARY SCHOOL,114.661177,Bayshore,727.299581
3,717,BEDOK,158,BEDOK STH AVE 3,158,BEDOK SOUTH AVENUE 3,SIGLAP EAST,460158,158 BEDOK SOUTH AVENUE 3 SIGLAP EAST SINGAPORE...,1985-11-01,BEDOK,EAST REGION,40438.709667,33444.903963,1.318736,103.945087,Tanah Merah,945.013991,MRT,Green,11100.213817,TEMASEK PRIMARY SCHOOL,126.857304,Bayshore,693.379599
4,718,BEDOK,159,BEDOK STH AVE 3,159,BEDOK SOUTH AVENUE 3,SIGLAP EAST,460159,159 BEDOK SOUTH AVENUE 3 SIGLAP EAST SINGAPORE...,1985-03-01,BEDOK,EAST REGION,40498.554980,33479.163810,1.319046,103.945624,Tanah Merah,903.863767,MRT,Green,11168.207858,TEMASEK PRIMARY SCHOOL,148.092394,Bayshore,751.697015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,5092,MARINE PARADE,58,MARINE TER,58,MARINE TERRACE,MARINE TERRACE HAVEN,440058,58 MARINE TERRACE MARINE TERRACE HAVEN SINGAPO...,1975-04-01,MARINE PARADE,CENTRAL REGION,36981.996568,32049.533872,1.306118,103.914026,Kembangan,1654.171105,MRT,Green,7375.299883,TAO NAN SCHOOL,290.196855,Marine Terrace,134.636019
75,5093,MARINE PARADE,59,MARINE TER,59,MARINE TERRACE,MARINE TERRACE HAVEN,440059,59 MARINE TERRACE MARINE TERRACE HAVEN SINGAPO...,1975-04-01,MARINE PARADE,CENTRAL REGION,37059.428064,32020.991769,1.305860,103.914722,Kembangan,1689.928823,MRT,Green,7439.174475,NGEE ANN PRIMARY SCHOOL,307.310317,Marine Terrace,98.828271
76,5094,MARINE PARADE,6,MARINE TER,6,MARINE TERRACE,MARINE TERRACE WALK,440006,6 MARINE TERRACE MARINE TERRACE WALK SINGAPORE...,1975-01-01,MARINE PARADE,CENTRAL REGION,37256.817208,31791.640995,1.303785,103.916495,Kembangan,1948.138081,MRT,Green,7557.101577,NGEE ANN PRIMARY SCHOOL,220.452984,Marine Terrace,353.940653
77,5095,MARINE PARADE,7,MARINE TER,7,MARINE TERRACE,MARINE TERRACE WALK,440007,7 MARINE TERRACE MARINE TERRACE WALK SINGAPORE...,1975-01-01,MARINE PARADE,CENTRAL REGION,37343.630417,31826.198648,1.304098,103.917275,Kembangan,1934.083413,MRT,Green,7650.190754,NGEE ANN PRIMARY SCHOOL,159.178638,Marine Terrace,372.601833
